6/14/20 This notebook worked with stack version 2020_11

In [30]:
import os
import numpy as np
import pandas as pd

from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
from lsst.daf.persistence import Butler

from lsst.sims.utils import angularSeparation

from ast import literal_eval

### Load the data needed

In [3]:
dd = pd.read_csv('raftInstall.csv',index_col=0)

In [4]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [18]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframe

In [80]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
properties = ['vendor', 'CCD', 'EO_run','ampX(pixel)', 'ampY(pixel)', 'ampX(deg)', 'ampY(deg)', 'fieldR(deg)', 'gain', 'readnoise', 
              'vignetting', 'effarea(m^2)'] + ['m5_%s'%i for i in filters]
amps = ()
for det in cam:
    rname, dname = det.getName().split('_')
    if len(rname)>3:
        continue
    if np.isnan(dd.run[rname]):
        print('no data yet, use default', rname, dname)
        continue
    for i in range(8):
        amps = amps + ('%s_%s_C0%d'%(rname,dname,i),)
    for i in range(8):
        amps = amps + ('%s_%s_C1%d'%(rname,dname,i),)
mdf = pd.DataFrame(index=amps, columns=properties, dtype=object)

no data yet, use default R00 SG0
no data yet, use default R00 SG1
no data yet, use default R04 SG0
no data yet, use default R04 SG1
no data yet, use default R40 SG0
no data yet, use default R40 SG1
no data yet, use default R44 SG0
no data yet, use default R44 SG1


### Fill the dataframe

In [103]:
for det in cam:
    rname, dname = det.getName().split('_')
    if len(rname)>3:
        continue
    if np.isnan(dd.run[rname]):
        print('no data yet, use default', rname, dname)
        continue
    vendor = det.getSerial()[:3]
    ccdname = det.getSerial()
    for iamp, amp in enumerate(det):
        i = amp.getName()
        idx = '%s_%s_%s'%(rname,dname,i)
        mdf.loc[idx, 'vendor'] = vendor
        mdf['CCD'].loc[idx]  = ccdname
        mdf['EO_run'][idx]  = int(dd.run[rname])
        amp_point = amp.getBBox().getCenter()
        mdf['ampX(pixel)'].loc[idx] = amp_point.x
        mdf['ampY(pixel)'].loc[idx] = amp_point.y
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [mdf['ampX(deg)'].loc[idx], mdf['ampY(deg)'].loc[idx]] = np.round(np.degrees(raDec),4)
        radius = angularSeparation(0., 0., mdf['ampX(deg)'][idx], mdf['ampY(deg)'][idx])
        mdf['fieldR(deg)'].loc[idx] = np.round(radius,4)
        mdf['gain'].loc[idx] = np.round(amp.getGain(),4)
        mdf['readnoise'].loc[idx] = np.round(amp.getReadNoise(),4)
        mdf['vignetting'].loc[idx] = np.round(np.interp(radius, vr, vv),4)
        mdf['effarea(m^2)'].loc[idx] = np.round(mdf['vignetting'][idx]*np.pi*(M1D/2)**2,3)
        
        #print(iamp, i)
        if iamp>7:
            ii = 15-iamp
        else:
            ii = iamp+8
        dfDir = os.path.join('m5_output', rname)
        
        #test we got the order right:
        #dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
        #df = pd.read_csv(dfPath, index_col=0)     
        #print(mdf['readnoise'].loc[idx], df[idx[:7]].apply(literal_eval)['readnoise'][ii] )
        
        dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
        df = pd.read_csv(dfPath, index_col=0)
        for f in filters:
            mdf['m5_%s'%f].loc[idx] = np.round(df[idx[:7]].apply(literal_eval)[f][ii],3)
#    break

no data yet, use default R00 SG0
no data yet, use default R00 SG1
no data yet, use default R04 SG0
no data yet, use default R04 SG1
no data yet, use default R40 SG0
no data yet, use default R40 SG1
no data yet, use default R44 SG0
no data yet, use default R44 SG1


In [104]:
mdf[:16]

,vendor,CCD,EO_run,ampX(pixel),ampY(pixel),ampX(deg),ampY(deg),fieldR(deg),gain,readnoise,vignetting,effarea(m^2),m5_u,m5_g,m5_r,m5_i,m5_z,m5_y
R01_S00_C00,ITL,ITL-3800C-083,10861,254,999.5,-1.0433,-1.7,1.9945,0.9678,6.6228,0.2227,12.224,23.184,24.203,23.798,23.387,22.831,21.929
R01_S00_C01,ITL,ITL-3800C-083,10861,763,999.5,-1.015,-1.7,1.9799,0.9784,6.5538,0.2518,13.822,23.303,24.298,23.881,23.468,22.909,22.005
R01_S00_C02,ITL,ITL-3800C-083,10861,1272,999.5,-0.9868,-1.7,1.9656,0.9841,6.7253,0.2776,15.238,23.373,24.361,23.94,23.526,22.966,22.062
R01_S00_C03,ITL,ITL-3800C-083,10861,1781,999.5,-0.9585,-1.7,1.9515,0.9828,6.8305,0.3036,16.665,23.445,24.423,23.996,23.58,23.019,22.115
R01_S00_C04,ITL,ITL-3800C-083,10861,2290,999.5,-0.9302,-1.7,1.9378,0.9712,6.7986,0.3295,18.087,23.52,24.482,24.049,23.631,23.068,22.163
R01_S00_C05,ITL,ITL-3800C-083,10861,2799,999.5,-0.9019,-1.7,1.9244,0.9713,6.7521,0.3552,19.497,23.588,24.535,24.096,23.677,23.111,22.207
R01_S00_C06,ITL,ITL-3800C-083,10861,3308,999.5,-0.8736,-1.7,1.9113,0.9723,6.4622,0.3722,20.43,23.658,24.582,24.134,23.711,23.143,22.237
R01_S00_C07,ITL,ITL-3800C-083,10861,3817,999.5,-0.8454,-1.7,1.8985,0.9554,6.154,0.3849,21.128,23.714,24.616,24.16,23.735,23.165,22.258
R01_S00_C10,ITL,ITL-3800C-083,10861,254,2999.5,-1.0433,-1.5889,1.9007,0.8919,6.1523,0.3827,21.007,23.705,24.61,24.154,23.732,23.166,22.256
R01_S00_C11,ITL,ITL-3800C-083,10861,763,2999.5,-1.015,-1.5889,1.8854,0.8923,6.2777,0.3985,21.874,23.734,24.634,24.178,23.755,23.188,22.277


In [38]:
189*16

3024

In [10]:
len(amps)

3024

In [105]:
dfPath = os.path.join('m5_output', 'm5_master_db.csv')
mdf.to_csv(dfPath)

In [100]:
dfPath

'm5_output/R43/m5_master_db.csv'

In [118]:
for f in filters:
    idx = mdf['m5_%s'%f]>0
    a = sum(idx)
    b = sum(mdf['m5_%s'%f]>-100)
    c = np.median(mdf['m5_%s'%f])
    d = np.median(mdf.loc[idx, 'm5_%s'%f])
    print(f, a,b,a/b, c, d)

u 3011 3024 0.9957010582010583 24.165 24.166
g 3011 3024 0.9957010582010583 24.951 24.952
r 3011 3024 0.9957010582010583 24.472 24.472
i 3011 3024 0.9957010582010583 24.004 24.004
z 3011 3024 0.9957010582010583 23.408 23.408
y 3011 3024 0.9957010582010583 22.467 22.467


In [120]:
mdf['vignetting'].max()

0.6225